# Preprocessing step

## Import of tools

In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

!pip install langdetect
from langdetect import detect

## Import cleaned datasets

In [9]:
game_reviews = pd.read_csv('/content/drive/MyDrive/Springboard DS/data/cleaned_game_reviews.csv', index_col=0)
game_ratings = pd.read_csv('/content/drive/MyDrive/Springboard DS/data/cleaned_game_ratings.csv', index_col=0)

In [10]:
game_reviews.describe()

,rating,ID
count,2.948623e+06,2.948623e+06
mean,6.886088e+00,8.918477e+04
std,1.796896e+00,9.072863e+04
min,1.401300e-45,1.000000e+00
25%,6.000000e+00,6.607000e+03
50%,7.000000e+00,4.277600e+04
75%,8.000000e+00,1.634120e+05
max,1.000000e+01,3.509920e+05


## Remove emoticons and digits

In [11]:
def emoticon_removal(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text) +
            ' '.join(emoticons).replace('-', ''))
    return text

game_reviews['comment'] = game_reviews['comment'].apply(emoticon_removal)

## Sample 20% of the data to make the dataframe small enough to work with

In [12]:
reviews_subset_df = game_reviews.sample(frac=0.2, replace=False, random_state=42)

## Identify language and remove non-English reviews


In [13]:
reviews_subset_df.describe()

,rating,ID
count,589725.000000,589725.000000
mean,6.884386,89281.868015
std,1.796214,90794.354853
min,0.001000,1.000000
25%,6.000000,6644.000000
50%,7.000000,42939.000000
75%,8.000000,163412.000000
max,10.000000,350992.000000


In [14]:
for idx, row in reviews_subset_df.iterrows():
  try:
    lang = detect(row['comment'])
    if lang != 'en':
      reviews_subset_df.drop(idx, axis=0, inplace=True)
  except:
    language = 'error'
    print("This row throws an error:", row)
    reviews_subset_df.drop(idx, axis=0, inplace=True)

# Outputting a csv at this stage so that I do not have to run the lengthy language process each time I run the worksheet
reviews_subset_df.to_csv('/content/drive/MyDrive/Springboard DS/data/english_only_game_reviews.csv')
reviews_subset_df.describe()

This row throws an error: rating                        7.3
comment    2016 5 8 6 7 8 7 7 7 8
ID                         176544
Name: 13285588, dtype: object
This row throws an error: rating                      5.0
comment    4 5 2 5 3 2 3 5 15 5
ID                       117914
Name: 15288017, dtype: object
This row throws an error: rating                         8.0
comment    3 5 3 5 3 5 4 3 5 18 25
ID                          258444
Name: 13660248, dtype: object
This row throws an error: rating                       7.0
comment    07 15 14 9 12 31 15 7
ID                         18905
Name: 17716328, dtype: object
This row throws an error: rating                         8.0
comment    4 3 5 4 3 5 4 5 19 5 25
ID                          136888
Name: 8078594, dtype: object
This row throws an error: rating                         7.0
comment    4 5 4 5 3 5 3 4 19 5 25
ID                          143986
Name: 10374456, dtype: object
This row throws an error: rating                     

,rating,ID
count,549447.000000,549447.000000
mean,6.868426,87775.651748
std,1.805478,90332.019783
min,0.001000,1.000000
25%,6.000000,6249.000000
50%,7.000000,41002.000000
75%,8.000000,161936.000000
max,10.000000,350992.000000


In [2]:
reviews_subset_df = pd.read_csv('/content/drive/MyDrive/Springboard DS/data/english_only_game_reviews.csv', index_col=0)
reviews_subset_df.describe()

,rating,ID
count,549447.000000,549447.000000
mean,6.868426,87775.651748
std,1.805478,90332.019783
min,0.001000,1.000000
25%,6.000000,6249.000000
50%,7.000000,41002.000000
75%,8.000000,161936.000000
max,10.000000,350992.000000


## Identify corpus-specific stopwords (from EDA)

In [3]:
bgg_stop_words = ['game', 'play', 'like', 'fun', 'one', 'good', 'really', 'great', 'much', 'also', 'interesting', 'many']

## Create stopwords set to use in CountVectorizer

In [4]:
full_set_stop_words = text.ENGLISH_STOP_WORDS.union(bgg_stop_words)

# Prepare test and training sets

Even with 20% of the original data, it's still too big to run through the vectorizer. Taking 50% of the remaining. 

In [5]:
reviews_sub_subset_df = reviews_subset_df.sample(frac=0.50, replace=False, random_state=42)

In [7]:
X = reviews_sub_subset_df.drop(['rating'], axis=1)
y = reviews_sub_subset_df['rating']

# For use with the count vectorizer
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X, y, test_size=0.33, random_state=42)

# For use with the TF-IDF vectorizer
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(X, y, test_size=0.33, random_state=42)

## Create features

### Count vectorization, with stop words passed in


In [15]:
bigram_count_vectorizer = CountVectorizer(ngram_range=(2, 2),
                                          decode_error='ignore', 
                                          min_df=20, max_df=0.95,
                                          token_pattern=r'\b[^\d\W]+\b',
                                          stop_words=full_set_stop_words)

# The token pattern includes the removal of digits
bigram_cv = bigram_count_vectorizer.fit_transform(X_train_cv['comment'])

In [16]:
trigram_count_vectorizer = CountVectorizer(ngram_range=(3, 3),
                                           decode_error='ignore',
                                           min_df=20, max_df=0.95,
                                           token_pattern=r'\b[^\d\W]+\b',
                                           stop_words=full_set_stop_words)

trigram_cv = trigram_count_vectorizer.fit_transform(X_train_cv['comment'])

In [17]:
bigram_columns = bigram_count_vectorizer.get_feature_names_out()

bigram_word_counts_cv = pd.DataFrame(bigram_cv.toarray(), 
                                     columns=bigram_columns)

bigram_word_counts_cv.head(5)

,_weight min,abilities cards,abilities make,ability cards,ability make,ability plan,ability players,ability use,able beat,able build,...,youtube com,yr old,z man,zero luck,zero player,zero sum,zombie apocalypse,zombie dice,zombie games,zombie theme
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
trigram_columns = trigram_count_vectorizer.get_feature_names_out()

trigram_word_counts_cv = pd.DataFrame(trigram_cv.toarray(),
                                      columns=trigram_columns)

trigram_word_counts_cv.head(5)

,abstract area control,abstract doesn t,abstract star star,abstract strategy games,abstract tile laying,accessible non gamers,action point allowance,action selection mechanic,action selection mechanism,actions don t,...,yahtzee style dice,year old daughter,year old enjoys,year old loves,year old s,year old son,year old year,years ago played,years don t,youtube com watch
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Export the CountVectorizer output for use in the modeling step

In [25]:
bigram_word_counts_cv.to_csv('/content/drive/MyDrive/Springboard DS/data/bigram_cv.csv')
trigram_word_counts_cv.to_csv('/content/drive/MyDrive/Springboard DS/data/trigram_cv.csv')

### TF-IDF vectorization, with stop words passed in

In [8]:
bigram_tfidf_vectorizer = TfidfVectorizer(ngram_range=(2, 2),
                                          decode_error='ignore',
                                          min_df=20, max_df=0.95,
                                          token_pattern=r'\b[^\d\W]+\b',
                                          stop_words=full_set_stop_words)

bigram_tf = bigram_tfidf_vectorizer.fit_transform(X_train_tf['comment'])

In [14]:
trigram_tfidf_vectorizer = TfidfVectorizer(ngram_range=(3, 3),
                                           decode_error='ignore',
                                           min_df=20, max_df=0.95,
                                           token_pattern=r'\b[^\d\W]+\b',
                                           stop_words=full_set_stop_words)

trigram_tf = trigram_tfidf_vectorizer.fit_transform(X_train_tf['comment'])

In [10]:
bigram_tf_columns = bigram_tfidf_vectorizer.get_feature_names_out()

bigram_word_counts_tf = pd.DataFrame(bigram_tf.toarray(),
                                     columns=bigram_tf_columns)

bigram_word_counts_tf.head(5)

,_weight min,abilities cards,abilities make,ability cards,ability make,ability plan,ability players,ability use,able beat,able build,...,youtube com,yr old,z man,zero luck,zero player,zero sum,zombie apocalypse,zombie dice,zombie games,zombie theme
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
trigram_tf_columns = trigram_tfidf_vectorizer.get_feature_names_out()

trigram_word_counts_tf = pd.DataFrame(trigram_tf.toarray(),
                                      columns=trigram_tf_columns)

trigram_word_counts_tf.head(5)

,abstract area control,abstract doesn t,abstract star star,abstract strategy games,abstract tile laying,accessible non gamers,action point allowance,action selection mechanic,action selection mechanism,actions don t,...,yahtzee style dice,year old daughter,year old enjoys,year old loves,year old s,year old son,year old year,years ago played,years don t,youtube com watch
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Export the TF-IDF output for use in the modeling step

In [16]:
bigram_word_counts_tf.to_csv('/content/drive/MyDrive/Springboard DS/data/bigram_tf.csv')
trigram_word_counts_tf.to_csv('/content/drive/MyDrive/Springboard DS/data/trigram_tf.csv')